In [1]:
import re
import numpy as np
import pandas as pd
import re
import geopandas as gpd
import os

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = (
    SparkSession.builder.appName("MAST30034 Project 2 Preprocessing")
    .config("spark.driver.memory", '4g')
    .config("spark.executor.memory", '8g')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.parquet.enableVectorizedReader","false")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.parquet.writeLegacyFormat", 'true')
    .getOrCreate()
)

22/09/26 19:04:12 WARN Utils: Your hostname, DESKTOP-3NQ3PQI resolves to a loopback address: 127.0.1.1; using 172.31.182.220 instead (on interface eth0)
22/09/26 19:04:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/26 19:04:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/26 19:04:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/26 19:04:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### Mapping Postcode to ABS Postal Areas

In [3]:
def postcode_to_str(col):
    return col.astype(str).str.zfill(4)

In [ ]:
# Read in data
postal_areas_gdf = gpd.read_file('../data/raw/postcodes/abs_postal_areas.zip')
consumer_details_df = pd.read_parquet('../data/curated/cleaned_consumers.parquet')
postcode_df = pd.read_csv('../data/raw/postcodes/postcodes.csv').drop_duplicates('postcode')

consumer_details_df['postcode'] = postcode_to_str(consumer_details_df['postcode'])
postcode_df['postcode'] = postcode_to_str(postcode_df['postcode'])

# Convert postcode dataframe to geodataframe
postcode_gdf = gpd.GeoDataFrame(
    postcode_df, geometry=gpd.points_from_xy(postcode_df['long'], postcode_df['lat'])
)
postcode_gdf.crs = postal_areas_gdf.crs

# Get list of postcodes not listed as abs postal areas and filter geodataframe to just these postcodes
unmapped = consumer_details_df[~consumer_details_df['postcode'].astype(str).str.zfill(4).isin(postal_areas_gdf['POA_CODE21'])]['postcode'].unique()
postcodes_gdf = postcode_gdf[postcode_gdf['postcode'].isin(unmapped)]

# Spatially join unmapped postcodes and abs postal areas
postcode_poa_gdf = postcodes_gdf.sjoin(postal_areas_gdf, how = 'inner')

# Remove and rename columns 
postcode_poa_df = postcode_poa_gdf[['postcode', 'POA_CODE21']]
postcode_poa_df = postcode_poa_df.rename(columns = {'POA_CODE21' : 'poa'})

# Combine abs mapped postcodes with unmapped postcodes
postcode_poa_df = pd.concat([postcode_poa_df, postal_areas_gdf[['POA_CODE21', 'POA_CODE21']].set_axis(['postcode', 'poa'], axis = 1)], ignore_index = True).reset_index(drop = True)

In [ ]:
output_dir = '../data/curated/census/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

postcode_poa_df.to_parquet(output_dir + 'postcode_poa.parquet', index = False)

All but 2 postcodes could be mapped to abs postal areas. Niether of these could be found in the Australia post website. https://postcodes-australia.com/postcodes/6958 says 6958 is a Western Australian postcode reserved for non standard use<br><br>
This results in the removal of 317 consumers from the dataset

In [ ]:
removed_consumers = consumer_details_df[~consumer_details_df['postcode'].astype(str).str.zfill(4).isin(postcode_poa_df['postcode'])]
len(removed_consumers), removed_consumers['postcode'].unique()

# Age/Gender Weighting

Read in data

In [108]:

age_df = pd.read_parquet(
    '../data/curated/census/age_data.parquet'
)

postcode_poa_df = pd.read_parquet(
    '../data/curated/census/postcode_poa.parquet'
)

Generate population by age intervals (18-24, 25-34, 35-44, 45-54, 55-64, 65+) at POA and national level

In [109]:
cols = []
for start_yr, end_yr in zip([18,25,35,45,55], [25,35,45,55,65]):
    for g in ['m', 'f', 'p']:
        col = f'age_{start_yr}_{end_yr - 1}_{g}'
        cols.append(col)
        age_df[col] = age_df.filter(regex = '|'.join([f'(age_yr_{x}_{g})'for x in range(start_yr,end_yr)])).astype(int).sum(axis = 1)

for g in ['m', 'f', 'p']:
    col = f'age_65+_{g}'
    cols.append(col)
    age_df[col] = age_df.filter(regex = '|'.join([f'(age_yr_{x}_{x+4}_{g})'for x in range(65,100, 5)] + ['age_yr_100_yr_over_[mf]'])).astype(int).sum(axis = 1)


age_df = age_df[['poa'] + cols].melt(id_vars = 'poa')

def get_gender(variable):
    g = variable[-1]
    if g == 'm':
        return 'Male'
    if g == 'f':
        return 'Female'
    if g == 'p':
        return 'Person'

age_df['gender'] = age_df['variable'].apply(get_gender)
age_df['variable'] = age_df['variable'].apply(lambda x : x[:-2])
age_df = pd.pivot_table(age_df, values = 'value', index =['poa', 'gender'], columns='variable')

idx = pd.IndexSlice
aus_age = age_df.loc[idx[:, 'Person'], :].sum()
aus_age_m = age_df.loc[idx[:, 'Male'], :].sum()
aus_age_f = age_df.loc[idx[:, 'Female'], :].sum()

aus_prob_m = aus_age_m.sum()/(aus_age.sum())

aus_age *= 1/aus_age.sum()
aus_age_m *= 1/aus_age_m.sum()
aus_age_f *= 1/aus_age_f.sum()

age_df = age_df.apply(lambda x : x/x.sum(), axis = 1)
age_df = age_df.fillna(age_df.mean())


Caculates weights for each combination of gender and postcode

In [110]:
# Constants for probability calculations
prob_bnpl = 0.05
prob_female_g_bnpl = 0.57
prob_male_g_bnpl = 0.43

prob_age_g_bnpl = pd.Series(data = {'age_18_24' : 0.26, 'age_25_34' : 0.35, 'age_35_44' : 0.2, 'age_45_54' : 0.12, 'age_55_64' : 0.04,'age_65+' : 0.01})

prob_bnpl_g_age_m = prob_age_g_bnpl*prob_male_g_bnpl*prob_bnpl/aus_age_m/aus_prob_m
prob_bnpl_g_age_f = prob_age_g_bnpl*prob_female_g_bnpl*prob_bnpl/aus_age_f/(1-aus_prob_m)
prob_bnpl_g_age = prob_age_g_bnpl*prob_bnpl/aus_age

def get_prob_bnpl(row):
    if row.name[1] == 'Male':
        return (row*prob_bnpl_g_age_m).sum()
    if row.name[1] == 'Female':
        return (row*prob_bnpl_g_age_f).sum()
    if row.name[1] == 'Person':
        return (row*prob_bnpl_g_age).sum()

age_df['weight'] = age_df.apply(get_prob_bnpl, axis = 1)
age_df = age_df[['weight']]


Transform index and write to disk

In [111]:
age_df = age_df.reset_index()
age_df['gender'] = age_df['gender'].apply(lambda x : 'Undisclosed' if x == 'Person' else x)

age_df.to_parquet('../data/curated/demographic_weights.parquet', index = False)